In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# unzip the datasets from gdrive into /content/
!unzip /content/drive/MyDrive/style-mate/datasets/clean_3class_320px_new.zip -d /content/

In [3]:
# !file /content/augmented/test/Heart/Heart__0_100.jpg

In [4]:
# Define constants
batch_size = 32
img_height = 320
img_width = 320
num_classes = 3
epochs = 20

datasets_dir = "/content/result"

# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.15,
    rotation_range=4,
    width_shift_range=0.04,
    height_shift_range=0.04
    )

## Load and augment training data
#train_data_gen = datagen.flow_from_directory(datasets_dir + 'train',
#                                             target_size=(img_height, img_width),
#                                             batch_size=batch_size,
#                                             class_mode='sparse')
#                                             #color_mode='grayscale')
#
## Load validation data
#val_data_gen = datagen.flow_from_directory(datasets_dir + 'test',
#                                           target_size=(img_height, img_width),
#                                           batch_size=batch_size,
#                                           class_mode='sparse')
#                                           #color_mode='grayscale')
#
## Load MobileNetV2 as a pretrained model
#base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
#                                               include_top=False,
#                                               weights='imagenet')
#base_model.trainable = False
#

train_generator = datagen.flow_from_directory(datasets_dir,
                                              target_size=(img_height, img_width),
                                              batch_size=batch_size,
                                              class_mode='sparse',
                                              subset='training'
                                              )

valid_generator = datagen.flow_from_directory(datasets_dir,
                                              target_size=(img_height, img_width),
                                              batch_size=batch_size,
                                              class_mode='sparse',
                                              subset='validation'
                                              )

Found 431 images belonging to 3 classes.
Found 74 images belonging to 3 classes.


In [14]:
# Create your model on top of MobileNetV2
#model = models.Sequential([
#    base_model,
#    layers.GlobalAveragePooling2D(),
#    layers.Dropout(0.25),
#    layers.Dense(25, activation='relu'),
#    layers.Dense(num_classes, activation='softmax')
#])
#
## Compile the model with sparse categorical entropy loss
#model.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])
#model=tf.keras.models.Sequential([
#        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(320, 320, 3)),
#        tf.keras.layers.MaxPooling2D(2, 2),
#        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#        tf.keras.layers.MaxPooling2D(2, 2),
#        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#        tf.keras.layers.MaxPooling2D(2, 2),
#        tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
#        tf.keras.layers.MaxPooling2D(2, 2),
#        tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
#        tf.keras.layers.MaxPooling2D(2, 2),
#        tf.keras.layers.Flatten(),
#        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=0.00003)),
#        tf.keras.layers.Dense(num_classes, activation='softmax')
#    ])

base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model.fit(training_generator, epochs=3)


In [15]:
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('/content/style_mate_image_classification.h5', save_weights_only=True, save_best_only=True)

In [16]:
# Train the model
model.fit(train_generator, epochs=25, validation_data=valid_generator) #, callbacks=[checkpoint_callback])


Epoch 1/25
14/14 [==============================] - 56s 4s/step - loss: 1.1189 - accuracy: 0.4385 - val_loss: 1.0467 - val_accuracy: 0.5270
Epoch 2/25
14/14 [==============================] - 52s 4s/step - loss: 0.9610 - accuracy: 0.5545 - val_loss: 1.1303 - val_accuracy: 0.5135
Epoch 3/25
14/14 [==============================] - 49s 4s/step - loss: 0.8331 - accuracy: 0.5986 - val_loss: 1.2157 - val_accuracy: 0.5541
Epoch 4/25
14/14 [==============================] - 52s 4s/step - loss: 0.7481 - accuracy: 0.6705 - val_loss: 1.2762 - val_accuracy: 0.5270
Epoch 5/25
14/14 [==============================] - 49s 4s/step - loss: 0.7012 - accuracy: 0.6937 - val_loss: 1.2535 - val_accuracy: 0.5405
Epoch 6/25
14/14 [==============================] - 48s 3s/step - loss: 0.6916 - accuracy: 0.6961 - val_loss: 1.2577 - val_accuracy: 0.5405
Epoch 7/25
14/14 [==============================] - 48s 3s/step - loss: 0.6837 - accuracy: 0.7030 - val_loss: 1.2914 - val_accuracy: 0.5676
Epoch 8/25
14/14 [==

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

# Define your label names based on your model's classes
class_labels = ['heart', 'oblong', 'square']

# Assuming you have your model defined as 'model'
img_path = '/content/heart_lmao.jpg'  # Change this to the path of your test image
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make a prediction
predictions = model.predict(img_array)

# Adjust the decoding logic based on the number of classes in your model
def decode_face_shape_predictions(predictions, class_labels):
    top_prediction_idx = np.argmax(predictions, axis=1)
    face_shape_prediction = class_labels[top_prediction_idx[0]]
    confidence = predictions[0][top_prediction_idx][0]
    return face_shape_prediction, confidence

# Decode predictions
predicted_face_shape, confidence = decode_face_shape_predictions(predictions, class_labels)

# Display the prediction
print(f"Predicted Face Shape: >> {predicted_face_shape} << with Confidence: {confidence:.2f}")


1/1 [==============================] - 0s 76ms/step
Predicted Face Shape: >> oblong << with Confidence: 0.57


In [23]:
tf.keras.saving.save_model(model, '/content/drive/MyDrive/style-mate/saved_model')

In [24]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/style-mate/saved_model')

In [25]:
tflite_model = converter.convert()

# Save the model.
with open('/content/drive/MyDrive/style-mate/style_mate_classification.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
!pip install tensorflowjs

In [28]:
!tensorflowjs_converter --input_format=keras  /content/drive/MyDrive/style-mate/style_mate_image_classification_new.h5 /content/drive/MyDrive/style-mate/tfjs_model/

2023-12-06 08:08:06.515195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 08:08:06.515267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 08:08:06.517104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 08:08:07.891805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
